<a href="https://colab.research.google.com/github/Cyril-19/gunew/blob/main/MNIST_dataset_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import tensorflow_datasets as tfds

2.9.2


In [42]:
mnist,mnist_info=tfds.load(name="mnist",with_info=True,as_supervised=True)

In [43]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='~/tensorflow_datasets/mnist/3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [44]:
mnist_train,mnist_test=mnist['train'],mnist['test']
validation_sample=.1*mnist_info.splits['train'].num_examples
validation_sample=tf.cast(validation_sample,tf.int64)
test_sample=mnist_info.splits['test'].num_examples
test_sample=tf.cast(test_sample,tf.int64)

In [45]:
def scale(image,label):
  image=tf.cast(image,tf.float32)
  image/=255
  return image,label

In [46]:
scaled_data=mnist['train'].map(scale)
scaled_test=mnist_test.map(scale)

In [47]:
BUFFER_SIZE = 10000
shuffled_data=scaled_data.shuffle(BUFFER_SIZE)
validation_data=shuffled_data.take(validation_sample)
train_data=shuffled_data.skip(validation_sample)
test_data=scaled_test.shuffle(BUFFER_SIZE)

In [48]:
BATCH_SIZE=100
train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(BATCH_SIZE)
test_data=test_data.batch(BATCH_SIZE)

validation_inputs, validation_targets = next(iter(validation_data))

In [49]:
validation_inputs, validation_targets 

(<tf.Tensor: shape=(100, 28, 28, 1), dtype=float32, numpy=
 array([[[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         ...,
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]]],
 
 
        [[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [

In [50]:
input=784
output=10
hidden_layer=50
model=tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer,activation='relu'),
                           tf.keras.layers.Dense(output, activation='softmax')
])

In [51]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [52]:
NUM_EPOCHS=5
model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5
540/540 - 8s - loss: 0.4262 - accuracy: 0.8810 - val_loss: 0.0867 - val_accuracy: 0.9800 - 8s/epoch - 15ms/step
Epoch 2/5
540/540 - 5s - loss: 0.1832 - accuracy: 0.9468 - val_loss: 0.0577 - val_accuracy: 0.9900 - 5s/epoch - 9ms/step
Epoch 3/5
540/540 - 5s - loss: 0.1355 - accuracy: 0.9599 - val_loss: 0.0403 - val_accuracy: 0.9900 - 5s/epoch - 9ms/step
Epoch 4/5
540/540 - 5s - loss: 0.1109 - accuracy: 0.9663 - val_loss: 0.0360 - val_accuracy: 0.9900 - 5s/epoch - 9ms/step
Epoch 5/5
540/540 - 5s - loss: 0.0949 - accuracy: 0.9717 - val_loss: 0.0343 - val_accuracy: 1.0000 - 5s/epoch - 9ms/step


In [53]:
test_loss, test_accuracy = model.evaluate(test_data)

100/100 [==============================] - 1s 3ms/step - loss: 0.1113 - accuracy: 0.9650


In [54]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.11. Test accuracy: 96.50%
